
# Part A):



In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
from transformers import BertTokenizer
import nltk

In [3]:
import en_core_web_sm

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from nltk.corpus import stopwords 

In [6]:
df = pd.read_json("https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json")
# set the display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

*What kind of data does it contain?* - Emails

In [7]:
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [8]:
#df

*How many entries does it have?* 11314 rows, 3 columns

*What is their distribution?*

In [9]:
df['target_names'].value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: target_names, dtype: int64

In [10]:
df.content[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

1st content matches the target name

Business question: trend analysis (hot topics), employee monitoring


In [11]:
df.head(1)

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos


In [12]:
data = df.content[0]

# Part B): Removing non-linguistic parts
Remove lines beginning with any of the following: ‘From:’, ‘Article-I.D.:’, ‘Organization:’, ‘Lines:’, ‘NNTP-Posting-Host:’, ‘Distribution:’, ‘Reply-T o:’, ‘X- Newsreader:’, ‘Expires:’ etc.

We're doing this to clean up the text data by removing metadata and other unnecessary information that may interfere with text analysis tasks. This will allow us to focus on the actual content of the text and extract meaningful insights from it.

In [13]:
lines = data.split('\n')
new_lines = []
for line in lines:
    if not re.match(r'^\s*(From:|Article-I.D.:|Organization:|Lines:|NNTP-Posting-Host:|Distribution:|Reply-To:|X-Newsreader:|Expires:|\s*-+\s*)', line, re.IGNORECASE):
        new_lines.append(line)

new_string = '\n'.join(new_lines)
data = re.sub(r'\b(?:Subject|Summary|Keywords):', '', new_string, flags=re.IGNORECASE)

print(data)


 WHAT car is this!?

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,







# Part C):

In [14]:
data = strip_numeric(data)
data = strip_punctuation(data)
data = strip_multiple_whitespaces(data)
print(data)

 WHAT car is this I was wondering if anyone out there could enlighten me on this car I saw the other day It was a door sports car looked to be from the late s early s It was called a Bricklin The doors were really small In addition the front bumper was separate from the rest of the body This is all I know If anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail Thanks 


strip_numeric: This function removes all
numeric characters from the text. It is useful when we want to treat numbers as noise or when the numbers are not relevant to our analysis.

strip_punctuation: This function removes all punctuation characters from the text. It is useful when we want to treat punctuation marks as noise or when the punctuation marks are not relevant to our analysis.

strip_multiple_whitespaces: This function removes all extra whitespace characters from the text and replaces them with a single space character. It is useful when we want to normalize the whitespace in the text and reduce it to a single space between words.

In [15]:
data = data.lower()
print(data)

 what car is this i was wondering if anyone out there could enlighten me on this car i saw the other day it was a door sports car looked to be from the late s early s it was called a bricklin the doors were really small in addition the front bumper was separate from the rest of the body this is all i know if anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail thanks 


# Part D): Remove stopwords and short words

In [16]:
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords

gensim_stopwords_sorted = sorted(list(gensim_stopwords))
nltk_stopwords_sorted = sorted(list(stopwords.words()))

print("Gensim Stopwords:\n", gensim_stopwords_sorted)
print("\nNLTK Stopwords:\n", nltk_stopwords_sorted)

Gensim Stopwords:
 ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'did', 'didn', 'do', 'does', 'doesn', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'former

Do you see a difference? Are negations included? Is that
reasonable?

Yes, there is a difference between the Gensim and NLTK stopword lists. The Gensim stopword list contains a smaller number of words (around 337) than the NLTK stopword list (around 179). Additionally, the specific words included in each list are different, as each list is compiled based on different criteria.

Regarding the inclusion of negations, neither the Gensim nor the NLTK stopword lists explicitly include negations. However, both lists contain words that could be considered negations in certain contexts, such as "not" or "no".

Whether it is reasonable to include negations in a stopword list depends on the specific use case and the goals of the text analysis. In some cases, removing negations could be beneficial for certain types of analyses, such as sentiment analysis. However, in other cases, keeping negations may be necessary for preserving the original meaning and intent of the text. Therefore, it is important to carefully consider the specific needs and goals of the analysis before deciding whether or not to include negations in the stopword list.

In [17]:
stopwords_to_remove = ["aren't",'isn', "isn't",'mightn', "mightn't",'mustn', "mustn't",
                      'needn',"needn't",'no', 'nor', 'not',"shan't",'shouldn', "shouldn't",
                      'wasn', "wasn't","weren't",'wouldn', "wouldn't"]

nltk_stopwords_adjusted = [word for word in stopwords.words() if word not in stopwords_to_remove]


print("\nAdjusted NLTK Stopwords:\n", sorted(nltk_stopwords_adjusted))


Adjusted NLTK Stopwords:
 ['a', 'a', 'a', 'a', 'a', 'a', 'aadi', 'aaj', 'aan', 'aap', 'aapne', 'aata', 'aati', 'aaya', 'aaye', 'ab', 'abans', 'abban', 'abbe', 'abbey', 'abbia', 'abbiamo', 'abbiano', 'abbiate', 'abe', 'aber', 'abhi', 'abia', 'able', 'about', 'about', 'above', 'above', 'accha', 'according', 'accordingly', 'acea', 'aceasta', 'această', 'aceea', 'aceeasi', 'acei', 'aceia', 'acel', 'acela', 'acelasi', 'acele', 'acelea', 'acest', 'acesta', 'aceste', 'acestea', 'acestei', 'acestia', 'acestui', 'aceşti', 'aceştia', 'acha', 'achcha', 'across', 'actually', 'ací', 'ad', 'ad', 'ada', 'adalah', 'adanya', 'adapun', 'adica', 'af', 'after', 'after', 'afterwards', 'again', 'again', 'against', 'against', 'agak', 'agaknya', 'agar', 'agar', 'agl', 'agli', 'ah', 'ahhoz', 'ahogy', 'ahol', 'ai', 'ai', 'ai', 'aia', 'aibă', 'aici', 'aie', 'aient', 'aies', 'ain', 'ain', "ain't", 'aint', 'aisa', 'aise', 'aisi', 'ait', 'aitzitik', 'així', 'això', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya',

In [18]:
# Remove stopwords
stopwords = nltk_stopwords_adjusted
data = remove_stopwords(data, stopwords=stopwords)
data

'car wondering enlighten car day sports car looked late early called bricklin doors small addition front bumper separate rest body tellme model engine specs years production car made history info funky car mail'

In [19]:
# Remove short words
data = strip_short(data)
data

'car wondering enlighten car day sports car looked late early called bricklin doors small addition front bumper separate rest body tellme model engine specs years production car made history info funky car mail'

In [20]:
# Stemming
data_stem = stem_text(data)
data_stem

'car wonder enlighten car dai sport car look late earli call bricklin door small addit front bumper separ rest bodi tellm model engin spec year product car made histori info funki car mail'

In [21]:
nlp = en_core_web_sm.load()

In [22]:
data_docs = []
for doc in nlp.pipe(data):
    data_docs.append(doc)

In [23]:
output = nlp(data)

In [24]:
print(output)

car wondering enlighten car day sports car looked late early called bricklin doors small addition front bumper separate rest body tellme model engine specs years production car made history info funky car mail


In [25]:
# Lemmatization
lemma_text = [token.lemma_ for token in output]
print(lemma_text)

['car', 'wonder', 'enlighten', 'car', 'day', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'tellme', 'model', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'car', 'mail']


In [26]:
data_lem = " ".join(lemma_text)
print(data_lem)

car wonder enlighten car day sport car look late early call bricklin door small addition front bumper separate rest body tellme model engine spec year production car make history info funky car mail


In [27]:
# Tokenization using BERT
bert_uncased = BertTokenizer.from_pretrained('bert-base-uncased')

In [28]:
print(len(list(bert_uncased.vocab.keys()))) #size of vocabulary

30522


In [29]:
print(list(bert_uncased.vocab.keys())[1997:2100])

['of', 'and', 'in', 'to', 'was', 'he', 'is', 'as', 'for', 'on', 'with', 'that', 'it', 'his', 'by', 'at', 'from', 'her', '##s', 'she', 'you', 'had', 'an', 'were', 'but', 'be', 'this', 'are', 'not', 'my', 'they', 'one', 'which', 'or', 'have', 'him', 'me', 'first', 'all', 'also', 'their', 'has', 'up', 'who', 'out', 'been', 'when', 'after', 'there', 'into', 'new', 'two', 'its', '##a', 'time', 'would', 'no', 'what', 'about', 'said', 'we', 'over', 'then', 'other', 'so', 'more', '##e', 'can', 'if', 'like', 'back', 'them', 'only', 'some', 'could', '##i', 'where', 'just', '##ing', 'during', 'before', '##n', 'do', '##o', 'made', 'school', 'through', 'than', 'now', 'years', 'most', 'world', 'may', 'between', 'down', 'well', 'three', '##d', 'year', 'while', 'will', '##ed', '##r']


In [30]:
data_BERT = bert_uncased.tokenize(data_stem)  
print(data_BERT[0])

car


In [31]:
# Save pre-processed data to GoogleDrive
from google.colab import drive
import os

folder_path = "/content/drive/MyDrive/PKL_Files"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)


with open(os.path.join(folder_path, "data_clean.pkl"), "wb") as f:
    pickle.dump(data, f)

with open(os.path.join(folder_path, "Stemmed.pkl"), "wb") as f:
    pickle.dump(data_stem, f)

with open(os.path.join(folder_path, "Lemma.pkl"), "wb") as f:
    pickle.dump(data_lem, f)

In [32]:
pickle_out = open("data.pickle", "wb")
pickle.dump(data, pickle_out)
pickle_out.close()

In [33]:
pickle_out = open("data_lem_joined.pickle", "wb")
pickle.dump(data_lem, pickle_out)
pickle_out.close()

In [34]:
pickle_out = open("data_stem", "wb")
pickle.dump(data_stem, pickle_out)
pickle_out.close()